In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np

# CA_01: Load the student dataset.
# The dataset is separated by semicolons, so we use sep=';'.
file_path = 'student-mat.csv'
df = pd.read_csv(file_path, sep=';')

# Display basic information about the dataframe
print("--- Initial Data Load ---")
print(f"Dataset shape: {df.shape}") # Shows (rows, columns)
print("\nFirst 5 rows of the dataset:")
print(df.head())
print("\nInformation about columns, types, and null values:")
df.info()

--- Initial Data Load ---
Dataset shape: (395, 33)

First 5 rows of the dataset:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]

Inform

In [3]:
# CA_02: Select specific numeric columns of interest.
columns_of_interest = ['age', 'studytime', 'failures', 'absences', 'G1', 'G2', 'G3']
df_selected = df[columns_of_interest].copy()

print("\n--- Selected Columns ---")
print("First 5 rows of the selected data:")
print(df_selected.head())

# CA_03: Filter and display students older than 18 years.
students_over_18 = df[df['age'] > 18].copy()

print("\n--- Students Older Than 18 ---")
print(f"Found {students_over_18.shape[0]} students older than 18.")
print(students_over_18.head())

# CA_04: Filter and display students with more than 20 absences.
students_high_absences = df[df['absences'] > 20].copy()

print("\n--- Students with More Than 20 Absences ---")
print(f"Found {students_high_absences.shape[0]} students with more than 20 absences.")
print(students_high_absences.head())


--- Selected Columns ---
First 5 rows of the selected data:
   age  studytime  failures  absences  G1  G2  G3
0   18          2         0         6   5   6   6
1   17          2         0         4   5   5   6
2   15          2         3        10   7   8  10
3   15          3         0         2  15  14  15
4   16          2         0         4   6  10  10

--- Students Older Than 18 ---
Found 29 students older than 18.
    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
127     GP   F   19       U     GT3       T     0     1   at_home     other   
153     GP   M   19       U     GT3       T     3     2  services   at_home   
210     GP   F   19       U     GT3       T     3     3     other     other   
247     GP   M   22       U     GT3       T     3     1  services  services   
257     GP   M   19       U     LE3       A     4     3  services   at_home   

     ... famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
127  ...      3        4 

In [4]:
print("--- TASK 2: Data Quality Check ---")
print(f"Initial number of rows: {df.shape[0]}")

# CA_05: Identify and remove duplicate records.
# First, let's count how many complete duplicate rows exist.
duplicate_rows = df.duplicated().sum()
print(f"\nFound {duplicate_rows} duplicate rows.")

# If there are duplicates, remove them.
if duplicate_rows > 0:
    df_cleaned = df.drop_duplicates().copy()
    print(f"Number of rows after removing duplicates: {df_cleaned.shape[0]}")
else:
    df_cleaned = df.copy()
    print("No duplicate rows to remove.")

# From now on, we will work with the cleaned dataframe.

--- TASK 2: Data Quality Check ---
Initial number of rows: 395

Found 0 duplicate rows.
No duplicate rows to remove.


In [5]:
# CA_07: Validate that grades (G1, G2, G3) are within the official range of 0-20.
# We can use the .describe() method for a quick statistical overview.
print("\n--- Grade Validation (G1, G2, G3) ---")
grade_stats = df_cleaned[['G1', 'G2', 'G3']].describe()
print(grade_stats)

# Check if any grade is outside the 0-20 range.
invalid_g1 = df_cleaned[(df_cleaned['G1'] < 0) | (df_cleaned['G1'] > 20)].shape[0]
invalid_g2 = df_cleaned[(df_cleaned['G2'] < 0) | (df_cleaned['G2'] > 20)].shape[0]
invalid_g3 = df_cleaned[(df_cleaned['G3'] < 0) | (df_cleaned['G3'] > 20)].shape[0]

if (invalid_g1 + invalid_g2 + invalid_g3) == 0:
    print("\nConfirmation: All grades are within the valid range of 0-20.")
else:
    print(f"\nWarning: Found {invalid_g1} invalid G1, {invalid_g2} invalid G2, {invalid_g3} invalid G3 grades.")


# CA_06: Detect outliers in 'absences' and final grades 'G3'.
# We'll use the Interquartile Range (IQR) method, a common statistical approach.
print("\n--- Outlier Detection (IQR Method) ---")

def detect_outliers(data, column_name):
    """Detects outliers in a specific column using the IQR method."""
    print(f"\nAnalyzing outliers for '{column_name}':")
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find the outliers
    outliers = data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)]
    
    print(f"Interquartile Range (IQR): {IQR}")
    print(f"Lower bound for outliers: {lower_bound:.2f}")
    print(f"Upper bound for outliers: {upper_bound:.2f}")
    print(f"Number of outliers detected: {len(outliers)}")
    
    if not outliers.empty:
        print("Example outliers found:")
        print(outliers.head())
    
    return outliers

# Detect outliers for 'absences' and 'G3'
outliers_absences = detect_outliers(df_cleaned, 'absences')
outliers_g3 = detect_outliers(df_cleaned, 'G3')


--- Grade Validation (G1, G2, G3) ---
               G1          G2          G3
count  395.000000  395.000000  395.000000
mean    10.908861   10.713924   10.415190
std      3.319195    3.761505    4.581443
min      3.000000    0.000000    0.000000
25%      8.000000    9.000000    8.000000
50%     11.000000   11.000000   11.000000
75%     13.000000   13.000000   14.000000
max     19.000000   19.000000   20.000000

Confirmation: All grades are within the valid range of 0-20.

--- Outlier Detection (IQR Method) ---

Analyzing outliers for 'absences':
Interquartile Range (IQR): 8.0
Lower bound for outliers: -12.00
Upper bound for outliers: 20.00
Number of outliers detected: 15
Example outliers found:
    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
40      GP   F   16       U     LE3       T     2     2     other     other   
74      GP   F   16       U     GT3       T     3     3     other  services   
103     GP   F   15       U     GT3       T     3     2 

In [6]:
print("--- TASK 3: Basic Performance Indicators ---")

# CA_07: Average, minimum, and maximum of the final grade (G3).
print("\n--- Final Grade (G3) Statistics ---")
avg_g3 = df_cleaned['G3'].mean()
min_g3 = df_cleaned['G3'].min()
max_g3 = df_cleaned['G3'].max()

print(f"Average Final Grade: {avg_g3:.2f}")
print(f"Minimum Final Grade: {min_g3}")
print(f"Maximum Final Grade: {max_g3}")

# CA_08: Number of students per study time level.
# The 'studytime' column is categorical: 1 (<2h), 2 (2-5h), 3 (5-10h), 4 (>10h).
print("\n--- Students per Study Time Level ---")
studytime_counts = df_cleaned['studytime'].value_counts().sort_index()
print(studytime_counts)
# .sort_index() is used to order the results by the study time level (1, 2, 3, 4).

# CA_09: General average of absences.
print("\n--- Absences Statistics ---")
avg_absences = df_cleaned['absences'].mean()
print(f"Average number of absences: {avg_absences:.2f}")

--- TASK 3: Basic Performance Indicators ---

--- Final Grade (G3) Statistics ---
Average Final Grade: 10.42
Minimum Final Grade: 0
Maximum Final Grade: 20

--- Students per Study Time Level ---
studytime
1    105
2    198
3     65
4     27
Name: count, dtype: int64

--- Absences Statistics ---
Average number of absences: 5.71
